# <strong><u>3a: Automated Gesture Annotation Notebook |</u></strong>

This notebook focuses on the annotation of gestures based on their speed profiles. It uses Pandas for data manipulation, NumPy for numerical operations, and Plotly for graphical representations.

The notebook reads in processed speed data for gestures and employs a function to generate annotations based on smoothed speed. These annotations include stroke phases and apex points within each gesture.

### <strong>Requirements</strong>

To run this notebook, you will need the following Python packages:

- pandas
- numpy
- plotly

You can install these packages using pip:
```shell
    pip install pandas numpy plotly
```

### <strong>Key Variables and Functions</strong>
- `MAX_S_LENGTH`, `MAX_STEADY_LENGTH`, `THRESHOLD`: Parameters controlling the annotation process
- `generate_annotations`: Function that generates stroke and apex annotations

By using this notebook, you'll be able to annotate gestures in a detailed manner, aiding in further analysis and research.



### <strong>Importing Libraries</strong>
- `pandas`: For DataFrame manipulation and data analysis
- `numpy`: For numerical computations
- `plotly`: For data visualization


In [12]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

### <strong> Load Data </strong>

In [13]:
data = pd.read_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/5003_I_board_one_right_wrist_processed_data.csv')
data.head()

,right_wrist_x,right_wrist_y,speed_unsmooth,speed_smooth,time_ms
0,0.310565,0.599996,NaN,NaN,0.0
1,0.310135,0.600007,NaN,NaN,10.0
2,0.309705,0.600017,NaN,NaN,20.0
3,0.309275,0.600028,0.038672,NaN,30.0
4,0.308915,0.599960,0.036750,NaN,40.0


### <strong>Variables</strong>

- `MAX_S_LENGTH`: an integer that represents the maximum length of an 'S' stroke in the gesture.
- `MAX_STEADY_LENGTH`: an integer that represents the maximum length of a steady phase in the gesture.
- `THRESHOLD`: a float that represents the threshold value for the smoothed speed of the gesture.
- `COLORS`: a dictionary that maps stroke phase annotations to RGBA color values.

These variables are used in the `generate_annotations` function to generate annotations for a gesture based on the smoothed speed of the gesture. The `MAX_S_LENGTH` and `MAX_STEADY_LENGTH` variables are used to determine the maximum length of an 'S' stroke and a steady phase, respectively. The `THRESHOLD` variable is used to determine when the gesture starts or ends a stroke or enters a steady phase. The `COLORS` variable is used to map stroke phase annotations to colors for visualization purposes.

Overall, this excerpt defines some important parameters for generating gesture annotations and demonstrates how variables can be defined and used in Python code.

In [14]:
MAX_S_LENGTH = 100
MAX_STEADY_LENGTH = 100
THRESHOLD = 0.4
COLORS = {'Stroke': 'rgba(231,107,243,0.2)', 'Rest': 'rgba(107,174,214,0.2)', '': 'rgba(255,255,255,0.2)'}

In [15]:
def generate_annotations(data): # Generate annotations based on speed_smooth data.
    speed_smooth = list(map(float, data['speed_smooth']))
    phase = [''] * len(speed_smooth)
    last_stroke_type = None
    start_index = None
    steady_start = None
    steady_end = None
    apexes = [''] * len(speed_smooth)

    # Iterate over the speed_smooth array to generate the annotations
    for i in range(1, len(speed_smooth)):
        if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if last_stroke_type is None or last_stroke_type == 'Steady' or last_stroke_type == 'R':
                last_stroke_type = 'Approach'
                start_index = i
            elif last_stroke_type == 'Approach' and i - start_index >= MAX_S_LENGTH:
                last_stroke_type = None
        elif speed_smooth[i] <= THRESHOLD and speed_smooth[i-1] > THRESHOLD:
            if last_stroke_type == 'Approach':
                last_stroke_type = 'Steady'
                start_index = i
                steady_start = i
            elif last_stroke_type == 'R':
                last_stroke_type = None
                start_index = None
        elif last_stroke_type == 'Steady' and speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if steady_end is not None:
                last_stroke_type = 'R'
                start_index = i

        if last_stroke_type == 'Steady' and steady_start is None:
            steady_start = i
        elif steady_start is not None and last_stroke_type != 'Steady':
            steady_end = i
            min_speed_index = np.argmin(speed_smooth[steady_start:i]) + steady_start
            apexes[min_speed_index] = 'AX'
            steady_start = None
            if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
                last_stroke_type = 'R'
                start_index = i
        elif last_stroke_type == 'Steady' and i - start_index >= MAX_STEADY_LENGTH:
            last_stroke_type = None
            start_index = None
            steady_start = None
        phase[i] = last_stroke_type if last_stroke_type is not None else ''

    data = pd.DataFrame({
        'time_ms': data['time_ms'],
        'speed_unsmooth': data['speed_unsmooth'],
        'speed_smooth': data['speed_smooth'],
        'phase': phase, 
        'apex': apexes
    })

    data['phase'] = data['phase'].apply(lambda x: 'Stroke' if x in ['Steady', 'Approach'] else ('Rest' if x == 'R' else ''))
    return data

data = generate_annotations(data)

### <strong>Visualize Annotations</strong>
The code first creates a `plotly` figure object and adds a scatter plot of the smoothed speed of the gesture over time. The code then adds rectangles to the plot to represent the stroke phases of the gesture. The rectangles are created by grouping the annotations by phase and finding the start and end times of each phase. The `COLORS` dictionary is used to map the stroke phase annotations to colors for the rectangles.

The code then adds scatter plots to the figure for each stroke phase and for the apex annotations. The `COLORS` dictionary is used to set the colors of the stroke phase markers.

Finally, the code updates the layout of the figure to include axis titles, a title, and a legend. The size of the figure is also set. The figure is then displayed using the `show` method.

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data['time_ms'], 
    y=data['speed_smooth'], 
    mode='lines',
    name='Speed curve'
))

data['phase_change'] = data['phase'].shift() != data['phase']
data['Group'] = data['phase_change'].cumsum()
interval_df = data.groupby(['Group', 'phase']).agg(Start=('time_ms', 'min'), End=('time_ms', 'max')).reset_index()

shapes = []
for _, row in interval_df.iterrows():
    shapes.append(dict(
        type="rect",
        xref="x", yref="paper",
        x0=row['Start'], x1=row['End'],
        y0=0, y1=1,
        fillcolor=COLORS[row['phase']],
        opacity=0.85,
        layer="below",
        line_width=0,
    ))

for phase in COLORS.keys():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=COLORS[phase]),
        showlegend=True,
        name=phase,
    ))

apex_df = data[data['apex'] == 'AX']
fig.add_trace(go.Scatter(
    x=apex_df['time_ms'], 
    y=apex_df['speed_smooth'], 
    mode='markers',
    name='Apexes'
))

fig.update_xaxes(title_text='Time (ms)')
fig.update_yaxes(title_text='Speed (px/ms)')
fig.update_layout(showlegend=True, shapes=shapes)
fig.update_layout(title_text='Smoothed Speed Curve with Annotations')
fig.update_layout(width=1250, height=750)
fig.show()


### <strong>Save Data</strong>

In [17]:
data = data.loc[:, ['time_ms', 'speed_smooth', 'phase', 'apex']]
data.to_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/ANNOTATIONS/5003_I_board_one_right_wrist_annotations.csv', index=False)
data.head()

,time_ms,speed_smooth,phase,apex
0,0.0,NaN,,
1,10.0,NaN,,
2,20.0,NaN,,
3,30.0,NaN,,
4,40.0,NaN,,
